In [39]:
import numpy as np
import matplotlib.pyplot as plt
import math

In [65]:
client_cost_per_piece = 0.5
our_part_per_piece = 0.7
salvage_rate = 0.1

machine = {
    'A': {
        'purchase_cost': 18000,
        'annual_maintenance_cost': 1350,
        'setup_cost': 750,
        'production_rate': 6,
        'internal_rate': 6*2
    },
    'B': {
        'purchase_cost': 27000,
        'annual_maintenance_cost': 2430,
        'setup_cost': 1000,
        'production_rate': 12,
        'interal_rate': int(12*1.5),
    },
    'C': {
        'purchase_cost': 64000,
        'annual_maintenance_cost': 4250,
        'setup_cost': 3000,
        'production_rate': 30,
        'internal_rate': 30,
    },
    'D': {
        'purchase_cost': 124000,
        'annual_maintenance_cost': 14400,
        'setup_cost': 6000,
        'production_rate': 30,
        'internal_rate': 30,
    },
}


Assumptions:<br>
- 200k pieces/year<br>
<br>because the machines have a 1M piece life cycle and the managers want a 5 year life cycle each year we need to produce 200K pieces for a total of 1M pieces after 5 years.
<br><br>

- salvage price of 10% of initial price<br>
<br>We analysed prices from [this website](https://www.sterlingmachinery.com/new-used-lathes-sale-engine-lathe/new-used-engine-lathe.html) and determined that the average resale price is close 0.1 times the original prices for those machines.
<br><br>

- The machines will run 2 shifts (8 hours each)<br>
<br>The reason for that is that we feel like it is useless to only utilize a 1/3 of a day to produce parts as it is a waste of time and can seriously slow down the project. It is also just common industry practice to have the machines run for more than 1 shift/day.
<br><br>

- 11% interest rate<br>
<br> we use that rate simply because it is very similar to the rates seen in practice problems throughout the course.
<br><br>

- Interest is compunded yearly (for simplicity's sake)

- 52 weeks x 5 days/week = 260 business days per year



In [53]:
base_hourly_salary = 14.29
extra_shift_multiplier = 0.25
benefits_multiplier = 0.63      # is applied to base pay only
manufacturing_overhead = 0.47   # is applied to direct pay

def nominal_labour_cost(days=260, shifts=2):
    '''for 1 operator'''

    hourly_cost1 = (
        base_hourly_salary + 
        base_hourly_salary * benefits_multiplier + 
        base_hourly_salary * manufacturing_overhead
    )
    shift1 = days * 8 * hourly_cost1

    if shifts > 1:

        extra_shifts = shifts-1
        direct_pay = base_hourly_salary * extra_shift_multiplier
        hourly_costx = (
            direct_pay + 
            base_hourly_salary * benefits_multiplier +
            direct_pay * manufacturing_overhead
        )
        shiftx = days * 8 * extra_shifts * hourly_costx
    else:
        shiftx = 0
    
    total_labour_cost = shift1 + shiftx

    return round(total_labour_cost,2)


In [84]:

def nominal_yearly_cost_client(machine_type, num_shifts=2, target=200000):
    num_machines, num_days = number_of_machines_required(machine_type, num_shifts, target)
    
    if machine_type == 'A' or machine_type == 'B':
        operators_required = num_machines
    elif machine_type == 'C':
        operators_required = num_machines//2
    else:
        operators_required = 0

    labour_cost = operators_required * nominal_labour_cost(num_days, num_shifts)

    cash_flow = {
        'negative':{
            '1time': [
                num_machines * machine[machine_type]['purchase_cost'], 
                num_machines * machine[machine_type]['setup_cost']
            ],
            'annual': [
                labour_cost, 
                num_machines * machine[machine_type]['annual_maintenance_cost'],
                client_cost_per_piece * target
            ]
        },
        'positive':{
            '1time': [salvage_rate * machine[machine_type]['purchase_cost'] * num_machines],
            'annual': []
        }
    }
    
    return cash_flow, num_machines, num_days


In [85]:
'''I want to plot machine A for 200k pieces'''
diagram = nominal_yearly_cost_client('A')
diagram

({'negative': {'1time': [162000, 6750],
   'annual': [739373.76, 12150, 100000.0]},
  'positive': {'1time': [16200.0], 'annual': []}},
 9,
 232)

In [43]:
def number_of_machines_required(machine_type, num_shifts, target=200000):
    '''
    This function tells you how many machines you need given a type and the number
    of shifts that the machines will be running (1, 2, or 3). It is assumed that machines run
    5 days/week, 4 weeks/month, 12 months/year.
    52 weeks x 5 days/week = 260 business days/year
    '''
    
    rate = machine[machine_type]['production_rate']
    N = math.ceil(target / (rate*num_shifts*8*5*52))
    # buffer = math.ceil(N) * rate * num_shifts * 8 * 5 * 4 * 12 - target
    days_required = math.ceil(target/ (N * rate * num_shifts * 8))
    return N, days_required

number_of_machines_required(machine_type='B', num_shifts=2, target=200000)

# make a pandas table to show the number of machines required and the days required

(5, 209)

In [7]:
needed_pieces = 8000
unit_price = 4.26
current_expenditures = needed_pieces * unit_price
current_expenditures 

34080.0